In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
import os
import crewai
from crewai import Agent, Task, Crew
from dotenv import find_dotenv, load_dotenv
import json

# _ = load_dotenv(find_dotenv())
load_dotenv("../../.env")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
CREW_SPEC = '../data/stock_trading_crew.json'
SRC_DIRECTORY = '../../data/ateam'
VERBOSE = True

In [2]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, DirectoryReadTool, FileReadTool
from crewai import Agent, Crew, Task
from crewai.tools import BaseTool
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

# Initialize the tools
tools = {
    "search": SerperDevTool(),
    "scrape": ScrapeWebsiteTool(),
    "directory_read": DirectoryReadTool(SRC_DIRECTORY),
    "file_read": FileReadTool(),
    'sentiment_analysis': SentimentAnalysisTool()
}


/Users/shankar/work/a.team/pocs/ai-agents/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/shankar/work/a.team/pocs/ai-agents/.venv/lib/python3.12/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
/Users/shankar/work/a.team/pocs/ai-agents/.venv/lib/python3.12/site-packages/crewai_tools/tools

In [3]:
with open(CREW_SPEC, "r", encoding="utf-8") as file:
    data = json.load(file)

In [4]:
def create_agent(agent_prop):
    agent = Agent(
        role=agent_prop["role"],
        goal=agent_prop["goal"],
        backstory=agent_prop["backstory"],
        verbose=VERBOSE,
        allow_delegation=agent_prop["allow_delegation"],
        tools=[tools[tool] for tool in agent_prop["tools"]] 
    )
    return agent

def create_task(task_prop, agents):
    task = Task(
        description=task_prop["description"],
        expected_output=task_prop["expected_output"],
        agent=agents[task_prop['agent']],
    )
    return task

def create_crew(agents, tasks):
    agent_list = [value for key, value in agents.items()]
    crew = Crew(
        agents=agent_list,
        
        tasks=tasks,
        
        manager_llm=ChatOpenAI(model="gpt-4o", 
                               temperature=0.7),
        process=Process.hierarchical,
        verbose=VERBOSE
    )

    return crew



In [5]:
agents_prop_list = data["agents"]
agents = {an_agent["agent"]: create_agent(an_agent) for an_agent in data["agents"]}
tasks = [create_task(task, agents) for task in data["tasks"]]

crew = create_crew(agents, tasks)


In [6]:
result = crew.kickoff(inputs=data["inputs"])

 
[2025-03-16 14:32:38][🚀 CREW 'CREW' STARTED, DFEE1113-255A-408A-B51B-F99F02655515]: 2025-03-16 14:32:38.247341
 
[2025-03-16 14:32:38][📋 TASK STARTED: CONTINUOUSLY MONITOR AND ANALYZE MARKET DATA FOR  THE SELECTED STOCK (AAPL). USE STATISTICAL MODELING AND MACHINE LEARNING TO IDENTIFY TRENDS AND PREDICT MARKET MOVEMENTS.]: 2025-03-16 14:32:38.258194
 
[2025-03-16 14:32:38][🤖 AGENT 'CREW MANAGER' STARTED TASK]: 2025-03-16 14:32:38.259195
# Agent: Crew Manager
## Task: Continuously monitor and analyze market data for  the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.
 
[2025-03-16 14:32:38][🤖 LLM CALL STARTED]: 2025-03-16 14:32:38.259454
 
[2025-03-16 14:32:41][✅ LLM CALL COMPLETED]: 2025-03-16 14:32:41.283120
 
[2025-03-16 14:32:41][🤖 TOOL USAGE STARTED: 'DELEGATE WORK TO COWORKER']: 2025-03-16 14:32:41.284069
 
[2025-03-16 14:32:41][🤖 AGENT 'DATA ANALYST' STARTED TASK]: 2025-03-16 14:32:41.286845
# Agent: Data An

In [7]:
from IPython.display import Markdown, display

In [8]:
Markdown(result.raw)

The current evaluation indicates that AAPL is trading at $213.49 and has experienced a year-to-date decline of 14.75%, yet the long-term outlook exudes investor confidence with significant returns over a five-year span. The trading strategies proposed for day trading with a medium risk tolerance are: Range Trading, Momentum Trading, Moving Averages & Indicators, Scalping, and the Gap and Go Strategy. Each comes with its risks given the current market condition and sentiment.

1. **Range Trading:**
   - **Risk:** AAPL exhibits significant volatility with a 52-week range of $164.08 to $260.10. There is a risk of price breaking the established range, leading to potential losses.
   - **Mitigation:** Monitor for breakout indicators strictly. Set stop-loss orders to minimize potential losses if the stock moves out of the expected range.

2. **Momentum Trading:**
   - **Risk:** Despite positive market sentiment, momentum can shift, leading to rapid loss if the stock’s price quickly reverses.
   - **Mitigation:** Use momentum indicators such as RSI or MACD. Develop a clear exit strategy and stay updated with market news that could indicate momentum shifts.

3. **Moving Averages & Indicators:**
   - **Risk:** Can result in late entry/exit from trades due to lagging nature of moving averages.
   - **Mitigation:** Combine with other leading indicators to confirm signals. Use shorter time frames for moving averages to respond quicker to price changes.

4. **Scalping:**
   - **Risk:** High transaction costs can erode profit margins, and rapid market changes can lead to small, frequent losses.
   - **Mitigation:** Ensure tight spreads by selecting an optimal trading platform. Limit scalping to times of sufficient market liquidity.

5. **Gap and Go Strategy:**
   - **Risk:** Relies heavily on early morning volatility, which can be unpredictable and lead to rapid losses.
   - **Mitigation:** Research pre-market movements and use news catalysts to predict direction. Implement pre-set stop-loss levels to manage unexpected moves.

**Additional Market Sentiment Insight:**
The overall sentiment leans negative among retail investors, according to TipRanks data, which indicates potential short-term downside risk. However, long-term ratings from analysts suggest a positive outlook, with some rating systems giving Apple a "Buy" rating based on market-beating probabilities.

To navigate these strategies effectively, ensure a robust risk management framework that includes diversification across strategies, regular review of market conditions, and real-time data analysis. Given the positive yet cautious market sentiment, cautious optimism should guide your trading decisions.

In [ ]:
def render_task_output(task):
    print(task.agent)
    Markdown(str(task.agent))
    print(str(task.description))
    print(str(task.raw))
[render_task_output(task) for task in result.tasks_output]

In [ ]:
result.tasks_output[0].agent

In [ ]:
result